In [5]:
import numpy as np
import pandas as pd
import math
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import roc_auc_score
import model_bias_analysis
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' # feel free to comment this line out if not using retina screen
import CommonFunctions

In [27]:
def train_and_predict(train_comments, test_comments, i):

    clf = Pipeline([
        ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2))),
        ('tfidf', TfidfTransformer(norm = 'l2')),
        ('clf', LogisticRegression()),
    ])

    clf = clf.fit(train_comments['comment'], train_comments['binary_tox'])
    auc = roc_auc_score(test_comments['binary_tox'], clf.predict_proba(test_comments['comment'])[:, 1])
    print('Test ROC AUC: %.3f' %auc)
    
    test_comments["predicted"+str(i)] = clf.predict(test_comments['comment'])
    
    return test_comments

In [30]:

def logDI(df, labels_col, terms):
    # labels_col is true/false of the comment being classified as toxic. ('binary_tox' I believe)
    # terms should be the array of top 8 terms
    
    logDI_arr = np.empty(((len(terms)), len(terms)))
    
    for i in range(len(terms)):
        for j in range(len(terms)):
            print("******", i)
            logDI_arr[i, j] =  (math.log(len(df[(df[terms[i]]==True) & (df[labels_col]==True)]) / len(df[df[terms[i]]==True])) \
                            - math.log(len(df[(df[terms[j]]==True) & (df[labels_col]==True)]) / len(df[df[terms[j]]==True])))**2
                
    logDI = logDI_arr.sum() / 2
    return logDI

In [32]:
madlibs_terms = ['gay', 'homosexual', 'straight', 'black', 'white', 'american', 'jewish', 'old']
dict_train_logs = {}
dict_test_logs = {}
array_predicted_bin_tox = []
array_test_bin_tox = []

#TODO: This code is really unnecessarily complicated, you could instead have a dictionary of dataframes and store all
for i in range(50, 55, 5):
# for i in range(30,40,5):
    joined_tox = pd.read_csv('joined_tox' + str(i) + '.csv')
    train_comments = pd.read_csv('train_comments'+ str(i) + '.csv')
    test_comments = pd.read_csv('test_comments' + str(i) + '.csv')
    # madlibs_terms = model_bias_analysis.read_identity_terms('test_comments'+str(i)+'.csv')
    
    log_val_train = logDI(train_comments, 'binary_tox', madlibs_terms)
    print(str(i), log_val_train)
    dict_train_logs[i] = log_val_train
    
    test_comments = train_and_predict(train_comments, test_comments,str(i))
    
#     temp_array = []
#     temp_array = test_comments['predicted'+str(i)]
#     array_predicted_bin_tox.append(temp_array)
    
#     temp_array = []
#     temp_array = test_comments['binary_tox']
#     array_test_bin_tox.append(temp_array)
    
    log_val_test = logDI(test_comments, 'predicted'+str(i), madlibs_terms)
    print(str(i), log_val_test)
    dict_test_logs[i] = log_val_test

****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 5
****** 5
****** 5
****** 5
****** 5
****** 5
****** 5
****** 5
****** 6
****** 6
****** 6
****** 6
****** 6
****** 6
****** 6
****** 6
****** 7
****** 7
****** 7
****** 7
****** 7
****** 7
****** 7
****** 7
50 25.29190818202314
Test ROC AUC: 0.953
****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 0
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 1
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 2
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 3
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 4
****** 5
****** 5
*****